In [1]:
#let's import all the packages we need
#requests: package used to query API and get the result back in Python
#json: package used to read and convert JSON format
#csv: package used to read and write csv
import requests,json,csv,os
import pandas as pd
import traceback

In [2]:
#Get movie ID from discover movies(popularity descending)
def get_ID(API_key, page):
    query = 'https://api.themoviedb.org/3/discover/movie?api_key='+API_key+'&language=en-US&sort_by=popularity.desc&include_video=false&page='+str(page)
    #Retrieve popularity descending 500 pages of MovieID data.
    response =  requests.get(query)
    if response.status_code==200: 
    #status code ==200 indicates the API query was successful
        array = response.json()
        return (array)
    else:
        return ("error")

In [3]:
#extract movie data as JSON
#Get movie data using Movie ID
def get_data(API_key, Movie_ID):
    query = 'https://api.themoviedb.org/3/movie/'+str(Movie_ID)+'?api_key='+API_key+'&language=en-US'
    #alters the movie id and api key for the url to retrieve the json data
    response =  requests.get(query)
    if response.status_code==200: 
    #status code ==200 indicates the API query was successful
        array = response.json()
        text = json.dumps(array)
        return (text)
    else:
        return ("error")

In [4]:
#unpack JSON into csv file
def write_file(filename, text):
    dataset = json.loads(text)
    csvFile = open(filename,'a', encoding='utf-8') #"a" - Append - Opens a file for appending, creates the file if it does not exist
    csvwriter = csv.writer(csvFile)
    try:
        #create a List from a dictionary under dataset['genres']
        genre_list = [x['name'] for x in dataset['genres']]
        production_company_list = [[x['name'],x['origin_country']] for x in dataset ['production_companies']] 
        production_country_list = [x['iso_3166_1'] for x in dataset['production_countries']]
        spoken_languages_list = [x['iso_639_1'] for x in dataset['spoken_languages']]
    except Exception:
        traceback.print_exc()
      
    result = [dataset['original_title'],dataset['adult'],genre_list,dataset['budget'],dataset['popularity'],dataset['vote_average'],dataset['vote_count'],dataset['overview'],dataset['tagline'],dataset['revenue'],dataset['release_date'],dataset['runtime'],dataset['original_language'],production_company_list,production_country_list,spoken_languages_list]
    # write data
    csvwriter.writerow(result)
    csvFile.close()

In [8]:
API_key = '33ba1424b719f150de8c4fdadc59f164'
Movie_ID = '464052' #wonderwoman
i=0

#Since getting data using individual Movie ID gives us the most data, 
#compared to getting data from a list of movies
#I extract 500 pages of movie ID from the most popular movies, and extract data 
#using a for-loop to write into the csv file for each get_data

page = [x for x in range(500,501)]
movie_id_list = []   
for p in page:
    Data = get_ID(API_key,p)
    if Data == "error":
        print("error")
    for y in Data["results"]:
        movie_id_list.append(y["id"])
print("movie list done\n")

#write header to the file
#creating the csv File with headers and name
csvFile = open('movie_collection_data_3.csv','a')
csvwriter = csv.writer(csvFile)
csvwriter.writerow(['Movie_name','Adult','GenreList','Budget','Popularity','VoteAvg','VoteCount','Overview','Tagline','Revenue','ReleaseDate','Runtime','OriginalLanguage', 'ProductionCompany','ProductionCountry','SpokenLanguage'])
csvFile.close()

for movie in movie_id_list:
    text = get_data(API_key, movie)
    #make sure your process breaks when the pull was not successful 
    #it's easier to debug this way
    if text == "error":
        break
    write_file('movie_collection_data_3.csv', text) #update csv
    print(str(i)) #to see status of data pull
    i +=1
    

movie list done

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
